In [1]:
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
from glob import glob
import re

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
marking = pd.read_csv('I:\\My Drive\\UAS_Beans\\Beans_StandCount\\2022\\SVREC\\ByStep\\d._Annotations\\Annot_beans_22_SVREC_7m__1.csv',index_col = False)
ROOT_IMG = 'I:\\My Drive\\UAS_Beans\\Beans_StandCount\\2022\\SVREC\\ByStep\\b._ClipPlots\\Plots_img'
#%cd /content/drive/MyDrive/MSU_beans/2022/Drone_pheno/SVREC/22_SVREC_RGB_7m_img/
marking

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,2201_1002_7m.png,4679526,"{""caption"":"""",""public_domain"":""no"",""image_url""...",119,0,"{""name"":""rect"",""x"":24,""y"":470,""width"":52,""heig...","{""bean_1"":{""1_plant"":true}}"
1,2201_1002_7m.png,4679526,"{""caption"":"""",""public_domain"":""no"",""image_url""...",119,1,"{""name"":""rect"",""x"":95,""y"":495,""width"":35,""heig...","{""bean_1"":{""1_plant"":true}}"
2,2201_1002_7m.png,4679526,"{""caption"":"""",""public_domain"":""no"",""image_url""...",119,2,"{""name"":""rect"",""x"":174,""y"":488,""width"":48,""hei...","{""bean_1"":{""1_plant"":true}}"
3,2201_1002_7m.png,4679526,"{""caption"":"""",""public_domain"":""no"",""image_url""...",119,3,"{""name"":""rect"",""x"":472,""y"":508,""width"":49,""hei...","{""bean_1"":{""1_plant"":true}}"
4,2201_1002_7m.png,4679526,"{""caption"":"""",""public_domain"":""no"",""image_url""...",119,4,"{""name"":""rect"",""x"":685,""y"":494,""width"":58,""hei...","{""bean_1"":{""1_plant"":true}}"
...,...,...,...,...,...,...,...
17255,2202_3008_7m.png,4722823,"{""caption"":"""",""public_domain"":""no"",""image_url""...",109,104,"{""name"":""rect"",""x"":3012,""y"":167,""width"":49,""he...","{""bean_1"":{""1_plant"":true}}"
17256,2202_3008_7m.png,4722823,"{""caption"":"""",""public_domain"":""no"",""image_url""...",109,105,"{""name"":""rect"",""x"":3238,""y"":182,""width"":19,""he...","{""bean_1"":{""1_plant"":true}}"
17257,2202_3008_7m.png,4722823,"{""caption"":"""",""public_domain"":""no"",""image_url""...",109,106,"{""name"":""rect"",""x"":2243,""y"":514,""width"":24,""he...","{""bean_1"":{""1_plant"":true}}"
17258,2202_3008_7m.png,4722823,"{""caption"":"""",""public_domain"":""no"",""image_url""...",109,107,"{""name"":""rect"",""x"":2272,""y"":513,""width"":62,""he...","{""bean_1"":{""1_plant"":true}}"


In [ ]:
## Showing images from directory and image format
img_list = os.listdir(ROOT_IMG)
img_list = [v for v in img_list if v.endswith('.png')]
print(img_list)
len(img_list)

['2201_1001_7m.png', '2201_1002_7m.png', '2201_1003_7m.png', '2201_1004_7m.png', '2201_1005_7m.png', '2201_1006_7m.png', '2201_1007_7m.png', '2201_1008_7m.png', '2201_1009_7m.png', '2201_1010_7m.png', '2201_1011_7m.png', '2201_1012_7m.png', '2201_1013_7m.png', '2201_1014_7m.png', '2201_1015_7m.png', '2201_1016_7m.png', '2201_1017_7m.png', '2201_1018_7m.png', '2201_1019_7m.png', '2201_1020_7m.png', '2201_1021_7m.png', '2201_1022_7m.png', '2201_1023_7m.png', '2201_1024_7m.png', '2201_1025_7m.png', '2201_1026_7m.png', '2201_1027_7m.png', '2201_1028_7m.png', '2201_1029_7m.png', '2201_1030_7m.png', '2201_1031_7m.png', '2201_1032_7m.png', '2201_1033_7m.png', '2201_1034_7m.png', '2201_1035_7m.png', '2201_1036_7m.png', '2201_2001_7m.png', '2201_2002_7m.png', '2201_2003_7m.png', '2201_2004_7m.png', '2201_2005_7m.png', '2201_2006_7m.png', '2201_2007_7m.png', '2201_2008_7m.png', '2201_2009_7m.png', '2201_2010_7m.png', '2201_2011_7m.png', '2201_2012_7m.png', '2201_2013_7m.png', '2201_2014_7m.png',

140

In [ ]:
coord_list = []
for img in img_list:
    lista = marking[marking.filename == img] 
    tamanho = lista.shape

    if tamanho[0] != 0:
        imagem = cv2.imread(os.path.join(ROOT_IMG, img)) 
        img_size = imagem.shape

        for i in range(tamanho[0]):
            line = lista.iloc[i,:] 
            region_shape = line.region_shape_attributes 
            region_attributes = (line.region_attributes)
            region_attributes = re.findall('"([^"]*)"', region_attributes) 
            coords = [float(s) for s in re.findall(r'-?\d+\.?\d*', region_shape)]
            x = int(coords[0]) 
            y = int(coords[1]) 
            w = int(coords[2]) 
            h = int(coords[3]) 
            
            coord_list.append([img.split('.')[0],img_size[1], img_size[0], x,y,w,h, "bean"]) ## saving coord

coord_list2 = np.array(coord_list)

print(coord_list2)



[['2201_1001_7m' '3894' '670' ... '23' '68' 'bean']
 ['2201_1001_7m' '3894' '670' ... '35' '51' 'bean']
 ['2201_1001_7m' '3894' '670' ... '25' '51' 'bean']
 ...
 ['2202_3008_7m' '3894' '671' ... '24' '56' 'bean']
 ['2202_3008_7m' '3894' '671' ... '62' '33' 'bean']
 ['2202_3008_7m' '3894' '671' ... '51' '31' 'bean']]


In [ ]:
# generating the Pandas dataframe
# from the Numpy array and specifying
# name of index and columns
panda_df = pd.DataFrame(data = coord_list2[0:, 0:],
                        index = [str(i + 1) 
                        for i in range(coord_list2.shape[0])],
                        columns = ['image_id', 'width', 'height', 'x','y','w','h','source'])
  
# printing the dataframe

panda_df

,image_id,width,height,x,y,w,h,source
1,2201_1001_7m,3894,670,14,450,23,68,bean
2,2201_1001_7m,3894,670,61,443,35,51,bean
3,2201_1001_7m,3894,670,113,461,25,51,bean
4,2201_1001_7m,3894,670,179,455,16,25,bean
5,2201_1001_7m,3894,670,220,453,25,49,bean
...,...,...,...,...,...,...,...,...
17256,2202_3008_7m,3894,671,3012,167,49,16,bean
17257,2202_3008_7m,3894,671,3238,182,19,16,bean
17258,2202_3008_7m,3894,671,2243,514,24,56,bean
17259,2202_3008_7m,3894,671,2272,513,62,33,bean


In [ ]:
## Saving in COCO annotations format
panda_df['bbox'] = panda_df[panda_df.columns[3:7]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

panda_df = panda_df.drop(panda_df.columns[3:7], axis=1)
panda_df['bbox'] = '[' + panda_df['bbox'].astype(str) + ']'

panda_df

,image_id,width,height,source,bbox
1,2201_1001_7m,3894,670,bean,"[14,450,23,68]"
2,2201_1001_7m,3894,670,bean,"[61,443,35,51]"
3,2201_1001_7m,3894,670,bean,"[113,461,25,51]"
4,2201_1001_7m,3894,670,bean,"[179,455,16,25]"
5,2201_1001_7m,3894,670,bean,"[220,453,25,49]"
...,...,...,...,...,...
17256,2202_3008_7m,3894,671,bean,"[3012,167,49,16]"
17257,2202_3008_7m,3894,671,bean,"[3238,182,19,16]"
17258,2202_3008_7m,3894,671,bean,"[2243,514,24,56]"
17259,2202_3008_7m,3894,671,bean,"[2272,513,62,33]"


In [ ]:
panda_df.to_csv('I:\\My Drive\\UAS_Beans\\Beans_StandCount\\2022\\SVREC\\ByStep\\d._Annotations\\Annot_beans_22_SVREC_7m__2.csv', index=False)   